In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
import seaborn as sns

In [80]:
train_data = pd.read_csv('data/england.csv')
test_data = pd.read_csv('data/future.csv')

In [4]:
train_data.head()

,date,time,day_of_week,season,country,league,league_level,home_team,away_tem,home_win_rate,draw_rate,away_win_rate,home_double_chance_rate,away_double_chance_rate,no_draw_rate,total_over_1_rate,total_under_1_rate,total_over_15_rate,total_under_15_rate,total_over_2_rate,total_under_2_rate,total_over_25_rate,total_under_25_rate,total_over_3_rate,total_under_3_rate,total_over_35_rate,total_under_35_rate,both_team_to_score_yes,both_team_to_score_no,link,home_scored,away_scored
0,28.12.2016,19:45,Wednesday,2016,england,premier-league-2016-2017,1,Southampton,Tottenham,3.03,3.20,2.61,1.54,1.42,1.38,1.07,7.9,1.35,3.29,1.55,2.52,2.19,1.77,3.20,1.37,3.93,1.26,1.86,2.02,https://www.oddsportal.com/soccer/england/prem...,1,4
1,08.04.2017,14:00,Saturday,2017,england,premier-league-2016-2017,1,Manchester City,Hull,1.19,7.96,16.00,1.02,5.40,1.10,0.00,0.0,1.09,7.40,1.09,6.80,1.35,3.26,1.50,2.66,1.93,1.96,1.95,1.93,https://www.oddsportal.com/soccer/england/prem...,3,1
2,17.12.2016,15:00,Saturday,2016,england,premier-league-2016-2017,1,Middlesbrough,Swansea,2.11,3.33,3.98,1.28,1.79,1.36,1.07,7.9,1.36,3.22,1.57,2.47,2.23,1.74,3.15,1.38,3.93,1.27,1.94,1.94,https://www.oddsportal.com/soccer/england/prem...,3,0
3,05.11.2016,15:00,Saturday,2016,england,premier-league-2016-2017,1,Manchester City,Middlesbrough,1.21,7.43,16.00,1.03,4.89,1.11,1.07,15.0,1.17,6.23,1.21,5.70,1.49,2.89,1.79,2.26,2.16,1.73,2.00,1.77,https://www.oddsportal.com/soccer/england/prem...,1,1
4,01.04.2017,16:30,Saturday,2017,england,premier-league-2016-2017,1,Southampton,Bournemouth,1.67,4.11,5.38,1.17,2.30,1.27,0.00,0.0,1.21,4.50,1.32,3.52,1.80,2.20,2.25,1.68,2.81,1.44,1.77,2.15,https://www.oddsportal.com/soccer/england/prem...,0,0


In [5]:
train_data = train_data.rename(columns={'away_tem':'away_team'})
test_data = test_data.rename(columns={'away_tem':'away_team'})

In [6]:
test_data.head()

,date,time,day_of_week,season,country,league,league_level,home_team,away_team,home_win_rate,draw_rate,away_win_rate,home_double_chance_rate,away_double_chance_rate,no_draw_rate,total_over_1_rate,total_under_1_rate,total_over_15_rate,total_under_15_rate,total_over_2_rate,total_under_2_rate,total_over_25_rate,total_under_25_rate,total_over_3_rate,total_under_3_rate,total_over_35_rate,total_under_35_rate,both_team_to_score_yes,both_team_to_score_no,link
0,10.03.2022,19:30,Today,2022,england,premier-league,1,Norwich,Chelsea,10.95,5.66,1.30,3.70,1.06,1.17,1.03,10.69,1.22,4.13,1.33,3.30,1.73,2.18,2.19,1.70,2.73,1.45,2.23,1.65,https://www.oddsportal.com/soccer/england/prem...
1,10.03.2022,19:30,Today,2022,england,premier-league,1,Southampton,Newcastle,2.00,3.73,3.79,1.29,1.89,1.30,1.05,9.25,1.24,3.89,1.37,3.07,1.81,2.08,2.33,1.61,2.89,1.40,1.68,2.18,https://www.oddsportal.com/soccer/england/prem...
2,10.03.2022,19:30,Today,2022,england,premier-league,1,Wolves,Watford,1.86,3.51,4.77,1.21,2.01,1.33,1.11,5.81,1.44,2.77,1.76,2.10,2.37,1.62,3.52,1.29,4.23,1.21,2.16,1.69,https://www.oddsportal.com/soccer/england/prem...
3,10.03.2022,19:45,Today,2022,england,premier-league,1,Leeds,Aston Villa,2.74,3.61,2.56,1.56,1.49,1.32,1.04,9.92,1.21,4.16,1.32,3.34,1.72,2.20,2.15,1.73,2.69,1.47,1.59,2.33,https://www.oddsportal.com/soccer/england/prem...
4,12.03.2022,12:30,Saturday,2022,england,premier-league,1,Brighton,Liverpool,6.20,4.61,1.52,2.65,1.14,1.22,1.02,11.29,1.20,4.41,1.29,3.54,1.66,2.28,2.06,1.79,2.58,1.50,1.75,2.05,https://www.oddsportal.com/soccer/england/prem...


In [7]:
train_data[train_data.total_over_1_rate.isna()]

,date,time,day_of_week,season,country,league,league_level,home_team,away_team,home_win_rate,draw_rate,away_win_rate,home_double_chance_rate,away_double_chance_rate,no_draw_rate,total_over_1_rate,total_under_1_rate,total_over_15_rate,total_under_15_rate,total_over_2_rate,total_under_2_rate,total_over_25_rate,total_under_25_rate,total_over_3_rate,total_under_3_rate,total_over_35_rate,total_under_35_rate,both_team_to_score_yes,both_team_to_score_no,link,home_scored,away_scored
950,26.12.2019,15:00,Thursday,2019,england,premier-league-2019-2020,1,Aston Villa,Norwich,2.23,3.79,3.04,1.40,1.69,1.29,NaN,11.0,1.16,5.12,1.22,4.0,1.53,2.50,1.82,2.04,2.32,1.61,1.46,2.74,https://www.oddsportal.com/soccer/england/prem...,1,0
959,27.12.2019,19:45,Friday,2019,england,premier-league-2019-2020,1,Wolves,Manchester City,7.07,5.52,1.40,3.11,1.12,1.17,NaN,15.0,1.15,5.35,1.22,4.4,1.48,2.69,1.77,2.13,2.26,1.65,1.67,2.19,https://www.oddsportal.com/soccer/england/prem...,3,2


In [8]:
train_data = train_data.fillna(1.01)

In [9]:
train_data.date = pd.to_datetime(train_data.date)

In [10]:
train_data.sort_values(by='date')

,date,time,day_of_week,season,country,league,league_level,home_team,away_team,home_win_rate,draw_rate,away_win_rate,home_double_chance_rate,away_double_chance_rate,no_draw_rate,total_over_1_rate,total_under_1_rate,total_over_15_rate,total_under_15_rate,total_over_2_rate,total_under_2_rate,total_over_25_rate,total_under_25_rate,total_over_3_rate,total_under_3_rate,total_over_35_rate,total_under_35_rate,both_team_to_score_yes,both_team_to_score_no,link,home_scored,away_scored
49,2016-01-10,14:00,Saturday,2016,england,premier-league-2016-2017,1,Sunderland,West Brom,2.58,3.20,3.19,1.39,1.56,1.39,1.08,7.20,1.37,3.10,1.65,2.30,2.35,1.71,3.40,1.34,4.10,1.24,1.85,1.95,https://www.oddsportal.com/soccer/england/prem...,1,1
116,2016-01-10,14:00,Saturday,2016,england,premier-league-2016-2017,1,Hull,Chelsea,7.70,5.00,1.46,3.02,1.10,1.21,1.04,10.50,1.18,4.84,1.27,3.92,1.70,2.34,2.15,1.83,2.63,1.49,1.78,2.04,https://www.oddsportal.com/soccer/england/prem...,0,2
153,2016-01-10,14:00,Saturday,2016,england,premier-league-2016-2017,1,Watford,Bournemouth,2.50,3.34,3.19,1.39,1.60,1.37,1.05,9.00,1.29,3.66,1.47,2.77,2.06,1.91,2.76,1.47,3.38,1.32,1.70,2.15,https://www.oddsportal.com/soccer/england/prem...,2,2
154,2016-01-10,14:00,Saturday,2016,england,premier-league-2016-2017,1,West Ham,Middlesbrough,2.08,3.50,4.06,1.27,1.83,1.34,1.06,8.00,1.30,3.54,1.50,2.68,2.10,1.87,2.86,1.45,3.52,1.30,1.78,2.04,https://www.oddsportal.com/soccer/england/prem...,1,1
92,2016-01-10,11:30,Saturday,2016,england,premier-league-2016-2017,1,Swansea,Liverpool,7.80,5.10,1.46,2.96,1.11,1.20,1.02,13.50,1.13,6.10,1.19,5.00,1.54,2.77,1.77,2.10,2.22,1.66,1.62,2.25,https://www.oddsportal.com/soccer/england/prem...,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1644,2022-12-02,15:00,Saturday,2022,england,premier-league,1,Watford,Brighton,3.93,3.36,2.09,1.83,1.27,1.35,1.10,6.66,1.40,3.00,1.67,2.27,2.24,1.69,3.21,1.34,4.04,1.24,1.91,1.90,https://www.oddsportal.com/soccer/england/prem...,0,2
1561,2022-12-02,15:00,Saturday,2022,england,premier-league,1,Everton,Leeds,2.44,3.78,2.80,1.49,1.57,1.31,1.04,10.02,1.21,4.48,1.28,3.59,1.65,2.29,2.10,1.80,2.62,1.50,1.55,2.45,https://www.oddsportal.com/soccer/england/prem...,3,0
1700,2022-12-02,12:30,Saturday,2022,england,premier-league,1,Manchester Utd,Southampton,1.58,4.42,5.81,1.14,2.57,1.23,1.03,12.79,1.18,4.91,1.23,4.06,1.56,2.47,1.88,2.00,2.38,1.60,1.63,2.29,https://www.oddsportal.com/soccer/england/prem...,1,1
1677,2022-12-02,17:30,Saturday,2022,england,premier-league,1,Norwich,Manchester City,17.14,7.97,1.18,4.72,1.04,1.12,1.02,13.47,1.17,5.01,1.22,4.21,1.53,2.55,1.85,2.03,2.33,1.63,2.39,1.58,https://www.oddsportal.com/soccer/england/prem...,0,4


In [11]:
numeric_features = train_data.select_dtypes(include=['int', 'float']).columns
numeric_features

Index(['season', 'league_level', 'home_win_rate', 'draw_rate', 'away_win_rate',
       'home_double_chance_rate', 'away_double_chance_rate', 'no_draw_rate',
       'total_over_1_rate', 'total_under_1_rate', 'total_over_15_rate',
       'total_under_15_rate', 'total_over_2_rate', 'total_under_2_rate',
       'total_over_25_rate', 'total_under_25_rate', 'total_over_3_rate',
       'total_under_3_rate', 'total_over_35_rate', 'total_under_35_rate',
       'both_team_to_score_yes', 'both_team_to_score_no', 'home_scored',
       'away_scored'],
      dtype='object')

In [12]:
date_features = ['date', 'time']

In [13]:
train_data = train_data.drop(columns=['link', 'league','country'])
test_data = test_data.drop(columns=['link', 'league', 'country'])

In [14]:
train_data = train_data.drop(columns=['league_level'])
test_data = test_data.drop(columns=['league_level'])

In [15]:
train_data['season'] = train_data.season.astype('object')
test_data['season'] = test_data.season.astype('object')

In [16]:
categorical_features = train_data.drop(columns=['date', 'time']).select_dtypes(include=['object']).columns
categorical_features

Index(['day_of_week', 'season', 'home_team', 'away_team'], dtype='object')

In [17]:
train_data.shape

(1740, 28)

In [18]:
def set_target(row):
    if row.home_scored > row.away_scored:
        return 0
    elif row.home_scored == row.away_scored:
        return 1
    else:
        return 2

In [19]:
train_data.head()

,date,time,day_of_week,season,home_team,away_team,home_win_rate,draw_rate,away_win_rate,home_double_chance_rate,away_double_chance_rate,no_draw_rate,total_over_1_rate,total_under_1_rate,total_over_15_rate,total_under_15_rate,total_over_2_rate,total_under_2_rate,total_over_25_rate,total_under_25_rate,total_over_3_rate,total_under_3_rate,total_over_35_rate,total_under_35_rate,both_team_to_score_yes,both_team_to_score_no,home_scored,away_scored
0,2016-12-28,19:45,Wednesday,2016,Southampton,Tottenham,3.03,3.20,2.61,1.54,1.42,1.38,1.07,7.9,1.35,3.29,1.55,2.52,2.19,1.77,3.20,1.37,3.93,1.26,1.86,2.02,1,4
1,2017-08-04,14:00,Saturday,2017,Manchester City,Hull,1.19,7.96,16.00,1.02,5.40,1.10,0.00,0.0,1.09,7.40,1.09,6.80,1.35,3.26,1.50,2.66,1.93,1.96,1.95,1.93,3,1
2,2016-12-17,15:00,Saturday,2016,Middlesbrough,Swansea,2.11,3.33,3.98,1.28,1.79,1.36,1.07,7.9,1.36,3.22,1.57,2.47,2.23,1.74,3.15,1.38,3.93,1.27,1.94,1.94,3,0
3,2016-05-11,15:00,Saturday,2016,Manchester City,Middlesbrough,1.21,7.43,16.00,1.03,4.89,1.11,1.07,15.0,1.17,6.23,1.21,5.70,1.49,2.89,1.79,2.26,2.16,1.73,2.00,1.77,1,1
4,2017-01-04,16:30,Saturday,2017,Southampton,Bournemouth,1.67,4.11,5.38,1.17,2.30,1.27,0.00,0.0,1.21,4.50,1.32,3.52,1.80,2.20,2.25,1.68,2.81,1.44,1.77,2.15,0,0


In [43]:
class DataTransformer:
    def __init__(self, context):
        self.train = context['train']
        self.test = context['test']
        self.numeric_features = context['numeric']
        self.categorical_features = context['categorical']
    
    def run_logic(self):
        self.train['target'] = self.train.apply(self.set_target, axis = 1)
        self.team_encoding()
        self.categorical_encoding()
        self.drop_useless_cols()
        self.split()
        return self.train, self.val, self.test
        
    
    def set_target(self, row):
        if row.home_scored > row.away_scored:
            return 0
        elif row.home_scored == row.away_scored:
            return 1
        else:
            return 2
        
    def drop_useless_cols(self):
        self.train = self.train.drop(columns=['date','time','home_scored','away_scored'])
        self.test = self.test.drop(columns=['date','time'])

        
    def all_time_mean(self):
        pass
    
    def season_mean(self):
        pass
    
    def team_encoding(self):
        all_teams = self.train.sort_values(by='home_team').home_team.unique()
        teams_labels = {team:number for number, team in enumerate(all_teams)}
        self.train['home_team'] = self.train['home_team'].map(teams_labels)
        self.train['away_team'] = self.train['away_team'].map(teams_labels)
        self.test['home_team'] = self.test['home_team'].map(teams_labels)
        self.test['away_team'] = self.test['away_team'].map(teams_labels)

    
    def categorical_encoding(self):
        labelencoder = LabelEncoder()
        for col in ['day_of_week', 'season']:
            self.train.loc[:, col] = labelencoder.fit_transform(self.train.loc[:, col])
            self.test.loc[:, col] = labelencoder.fit_transform(self.test.loc[:, col])
        
    
    
    def season_current_pos(self):
        pass
    
    def split(self):
        train_shape = self.train.shape
        self.val = self.train.loc[train_shape[0] - 150:train_shape[0] - 51]
        self.test = self.train.loc[train_shape[0] - 50:]

        self.train = self.train.loc[:train_shape[0] - 151]


In [44]:
context = {'train':train_data,
            'test':test_data,
           'numeric':numeric_features,
           'categorical':categorical_features
          }

In [45]:
transformer = DataTransformer(context)
train, val, test = transformer.run_logic()

In [46]:
train.shape, val.shape, test.shape

((1590, 25), (100, 25), (50, 25))

In [52]:
X_train, y_train = train.drop(columns=['target']), train.target
X_val, y_val = val.drop(columns=['target']), val.target
X_test, y_test = test.drop(columns=['target']), test.target

# Baseline model

In [53]:
from catboost import CatBoostClassifier, Pool

In [54]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score

In [55]:
cat_features = list(categorical_features)

Train = Pool(data=X_train,
             label=y_train,
             cat_features=cat_features)
            
Valid = Pool(data=X_val,
             label=y_val,
             cat_features=cat_features)

In [56]:
model = CatBoostClassifier( random_seed = 17,     
                           n_estimators = 1000,
                           learning_rate = 0.01,
                            thread_count = -1, 
                            verbose = 100,  
                            loss_function='MultiClass',
                            task_type = "GPU",
                          early_stopping_rounds=100)


model.fit(Train, eval_set=Valid)
preds_class = model.predict(Valid)
preds_proba = model.predict_proba(Valid)

0:	learn: 1.0962077	test: 1.0957340	best: 1.0957340 (0)	total: 8.04ms	remaining: 8.03s
100:	learn: 0.9491187	test: 0.9551923	best: 0.9551923 (100)	total: 599ms	remaining: 5.33s
200:	learn: 0.9026908	test: 0.9230653	best: 0.9230653 (200)	total: 1.52s	remaining: 6.05s
300:	learn: 0.8770466	test: 0.9161066	best: 0.9160754 (299)	total: 2.07s	remaining: 4.81s
400:	learn: 0.8583060	test: 0.9161784	best: 0.9151221 (334)	total: 2.62s	remaining: 3.92s
bestTest = 0.9151220703
bestIteration = 334
Shrink model to first 335 iterations.


In [57]:
print(f'Validation ACCURACY SCORE: {accuracy_score(y_val, preds_class)}')

Validation ACCURACY SCORE: 0.59


In [58]:
print(f'Validation ROC AUC SCORE: {roc_auc_score(y_val, preds_proba, multi_class="ovr")}')

Validation ROC AUC SCORE: 0.7007206005713469


In [59]:
roc_auc_score(y_val, preds_proba, multi_class='ovr')

0.7007206005713469

In [60]:
test_proba = model.predict_proba(X_test)

In [61]:
test

,day_of_week,season,home_team,away_team,home_win_rate,draw_rate,away_win_rate,home_double_chance_rate,away_double_chance_rate,no_draw_rate,total_over_1_rate,total_under_1_rate,total_over_15_rate,total_under_15_rate,total_over_2_rate,total_under_2_rate,total_over_25_rate,total_under_25_rate,total_over_3_rate,total_under_3_rate,total_over_35_rate,total_under_35_rate,both_team_to_score_yes,both_team_to_score_no,target
1690,0,5,3,0,4.01,3.34,2.07,1.83,1.27,1.36,1.11,5.94,1.42,2.87,1.73,2.19,2.28,1.65,3.32,1.32,4.14,1.23,1.97,1.84,0
1691,3,5,29,15,4.52,4.27,1.73,2.20,1.22,1.25,1.02,13.83,1.16,5.17,1.20,4.44,1.50,2.63,1.76,2.14,2.24,1.68,1.51,2.54,0
1692,2,5,27,19,2.14,3.51,3.56,1.32,1.77,1.33,1.06,8.57,1.29,3.62,1.46,2.77,1.94,1.94,2.62,1.51,3.22,1.35,1.73,2.12,1
1693,2,6,4,5,1.67,3.79,5.86,1.16,2.24,1.31,1.08,6.87,1.35,3.27,1.58,2.43,2.13,1.77,2.93,1.40,3.65,1.28,2.02,1.80,2
1694,2,5,3,20,1.64,4.02,5.74,1.15,2.38,1.27,1.07,8.24,1.30,3.51,1.47,2.73,1.95,1.93,2.67,1.49,3.30,1.34,1.94,1.89,2
1695,2,6,8,5,1.91,3.40,4.63,1.22,1.94,1.36,1.11,6.29,1.42,2.90,1.73,2.19,2.27,1.65,3.31,1.32,4.10,1.23,2.04,1.79,1
1696,2,5,5,8,2.82,3.25,2.70,1.49,1.46,1.38,1.11,6.33,1.40,3.00,1.71,2.22,2.25,1.67,3.29,1.33,4.08,1.23,1.89,1.93,1
1697,1,5,29,14,2.46,3.60,2.86,1.44,1.59,1.33,1.05,9.25,1.24,4.05,1.35,3.20,1.78,2.11,2.27,1.66,2.83,1.44,1.60,2.34,0
1698,3,5,15,16,2.98,3.69,2.37,1.62,1.43,1.32,1.04,9.69,1.22,4.30,1.32,3.36,1.72,2.21,2.17,1.73,2.70,1.48,1.58,2.40,1
1699,1,6,26,9,1.73,3.90,5.07,1.18,2.22,1.29,1.05,9.35,1.24,4.07,1.37,3.10,1.78,2.11,2.32,1.63,2.92,1.41,1.70,2.15,0


In [63]:
test_proba[:10]

array([[0.26420967, 0.32941149, 0.40637885],
       [0.22966717, 0.24172093, 0.52861191],
       [0.41911256, 0.28001365, 0.30087379],
       [0.55193217, 0.23288228, 0.21518555],
       [0.59808297, 0.20293859, 0.19897844],
       [0.45823987, 0.25907137, 0.28268875],
       [0.32511229, 0.32508986, 0.34979785],
       [0.35597813, 0.28083924, 0.36318263],
       [0.35024672, 0.25112438, 0.3986289 ],
       [0.59442683, 0.22578487, 0.1797883 ]])

In [81]:
all_teams = train_data.sort_values(by='home_team').home_team.unique()
teams_labels = {team:number for number, team in enumerate(all_teams)}
decode_teams = {number:team for number, team in enumerate(all_teams)}

In [82]:
test_predictions = pd.DataFrame()
test_predictions['home_team'] = test.home_team.map(decode_teams)
test_predictions['away_team'] = test.away_team.map(decode_teams)
test_predictions['home_win_proba'] = test_proba[:,0]
test_predictions['draw_proba'] = test_proba[:,1]
test_predictions['away_win_proba'] = test_proba[:,2]
test_predictions['home_win_rate'] = test.home_win_rate
test_predictions['draw_rate'] = test.draw_rate
test_predictions['away_win_rate'] = test.away_win_rate
test_predictions['result'] = test.target


In [83]:
test_predictions

,home_team,away_team,home_win_proba,draw_proba,away_win_proba,home_win_rate,draw_rate,away_win_rate,result
1690,Brentford,Arsenal,0.264210,0.329411,0.406379,4.01,3.34,2.07,0
1691,West Ham,Liverpool,0.229667,0.241721,0.528612,4.52,4.27,1.73,0
1692,Watford,Newcastle,0.419113,0.280014,0.300874,2.14,3.51,3.56,1
1693,Brighton,Burnley,0.551932,0.232882,0.215186,1.67,3.79,5.86,2
1694,Brentford,Norwich,0.598083,0.202939,0.198978,1.64,4.02,5.74,2
1695,Crystal Palace,Burnley,0.458240,0.259071,0.282689,1.91,3.40,4.63,1
1696,Burnley,Crystal Palace,0.325112,0.325090,0.349798,2.82,3.25,2.70,1
1697,West Ham,Leicester,0.355978,0.280839,0.363183,2.46,3.60,2.86,0
1698,Liverpool,Manchester City,0.350247,0.251124,0.398629,2.98,3.69,2.37,1
1699,Tottenham,Everton,0.594427,0.225785,0.179788,1.73,3.90,5.07,0


# Считаем ROI

In [101]:
class ROIChecker():
    def __init__(self, predictions):
        self.predictions = predictions
        self.roi_info = pd.DataFrame()

    
    def get_roi(self):
        
        self.predictions['home_win_ROI'] = self.predictions.home_win_rate * self.predictions.home_win_proba - 1
        self.predictions['away_win_ROI'] = self.predictions.away_win_rate * self.predictions.away_win_proba - 1
        self.predictions['draw_ROI'] = self.predictions.draw_rate * self.predictions.draw_proba - 1
        
        
        return self.predictions

In [102]:
checker = ROIChecker(test_predictions)
roi_info = checker.get_roi()

In [103]:
roi_info.head()

,home_team,away_team,home_win_proba,draw_proba,away_win_proba,home_win_rate,draw_rate,away_win_rate,result,home_win_ROI,away_win_ROI,draw_ROI
1690,Brentford,Arsenal,0.264210,0.329411,0.406379,4.01,3.34,2.07,0,0.059481,-0.158796,0.100234
1691,West Ham,Liverpool,0.229667,0.241721,0.528612,4.52,4.27,1.73,0,0.038096,-0.085501,0.032148
1692,Watford,Newcastle,0.419113,0.280014,0.300874,2.14,3.51,3.56,1,-0.103099,0.071111,-0.017152
1693,Brighton,Burnley,0.551932,0.232882,0.215186,1.67,3.79,5.86,2,-0.078273,0.260987,-0.117376
1694,Brentford,Norwich,0.598083,0.202939,0.198978,1.64,4.02,5.74,2,-0.019144,0.142136,-0.184187


In [104]:
def explain_roi_info(roi_info):
    roi_cols = ['home_win_ROI', 'away_win_ROI', 'draw_ROI']
    best_roi_df = pd.DataFrame()
    for index, row in roi_info.iterrows():
        print(f"Match #{index+1}: {row.home_team} vs {row.away_team}")
        
        max_roi = np.max(roi_info.loc[index, roi_cols])
        current_choice = 'home_win_ROI'
        
        for col in roi_cols:
            if row[col] == max_roi:
                current_choice = col
        current_choice = ' '.join(current_choice.split('_')[:1])
        
        if max_roi > 0:
            print(f"Maximal ROI = {max_roi*100}% on {current_choice}")
        
        else:
            print("Нет положительного ROI")
            
        print('_______________________________ \n')
        
        best_roi_df.loc[index, 'home_team'] = row.home_team
        best_roi_df.loc[index, 'away_team'] = row.away_team
        best_roi_df.loc[index, 'best_ROI'] = max_roi
        best_roi_df.loc[index, 'choice'] = current_choice
        best_roi_df.loc[index, 'home_win_rate'] = row.home_win_rate
        best_roi_df.loc[index, 'draw_rate'] = row.draw_rate
        best_roi_df.loc[index, 'away_win_rate'] = row.away_win_rate
        best_roi_df.loc[index, 'result'] = row.result
        
    return best_roi_df

In [105]:
best_roi_df = explain_roi_info(roi_info)

Match #1691: Brentford vs Arsenal
Maximal ROI = 10.02343634845484% on draw
_______________________________ 

Match #1692: West Ham vs Liverpool
Maximal ROI = 3.809558798574697% on home
_______________________________ 

Match #1693: Watford vs Newcastle
Maximal ROI = 7.111068368387086% on away
_______________________________ 

Match #1694: Brighton vs Burnley
Maximal ROI = 26.09873126218858% on away
_______________________________ 

Match #1695: Brentford vs Norwich
Maximal ROI = 14.21362367515382% on away
_______________________________ 

Match #1696: Crystal Palace vs Burnley
Maximal ROI = 30.884893304644013% on away
_______________________________ 

Match #1697: Burnley vs Crystal Palace
Maximal ROI = 5.654205839120863% on draw
_______________________________ 

Match #1698: West Ham vs Leicester
Maximal ROI = 3.8702331254298006% on away
_______________________________ 

Match #1699: Liverpool vs Manchester City
Maximal ROI = 4.373522633245441% on home
_______________________________ 

In [106]:
best_roi_df

,home_team,away_team,best_ROI,choice,home_win_rate,draw_rate,away_win_rate,result
1690,Brentford,Arsenal,0.100234,draw,4.01,3.34,2.07,0.0
1691,West Ham,Liverpool,0.038096,home,4.52,4.27,1.73,0.0
1692,Watford,Newcastle,0.071111,away,2.14,3.51,3.56,1.0
1693,Brighton,Burnley,0.260987,away,1.67,3.79,5.86,2.0
1694,Brentford,Norwich,0.142136,away,1.64,4.02,5.74,2.0
1695,Crystal Palace,Burnley,0.308849,away,1.91,3.40,4.63,1.0
1696,Burnley,Crystal Palace,0.056542,draw,2.82,3.25,2.70,1.0
1697,West Ham,Leicester,0.038702,away,2.46,3.60,2.86,0.0
1698,Liverpool,Manchester City,0.043735,home,2.98,3.69,2.37,1.0
1699,Tottenham,Everton,0.028358,home,1.73,3.90,5.07,0.0


In [111]:
def money_score(best_roi_df, bet=100):
    results = {'home':0, 'draw':1, 'away':2}
    cols = {'home':'home_win_rate', 'draw':'draw_rate', 'away':'away_win_rate'}
    profit = 0
    
    for index, row in best_roi_df.iterrows():
        if results[row.choice] == row.result:
            current_profit = bet * (row[cols[row.choice]] - 1)
            profit += current_profit
            print(row.choice, row[cols[row.choice]], current_profit)
        else:
            profit -= bet
    return profit
    

In [112]:
money_score(best_roi_df, 100)

home 4.52 351.99999999999994
away 5.86 486.00000000000006
away 5.74 474.0
draw 3.25 225.0
home 1.73 73.0
draw 4.42 342.0
draw 3.31 231.0
home 4.08 308.0
away 2.5 150.0
draw 3.35 235.0
away 5.66 466.0
away 2.19 119.0
draw 3.54 254.0
home 1.53 53.0


168.0

# Наброски фичей

In [65]:
train_data.groupby(['home_team'],as_index=False).mean().head()

,home_team,home_win_rate,draw_rate,away_win_rate,home_double_chance_rate,away_double_chance_rate,no_draw_rate,total_over_1_rate,total_under_1_rate,total_over_15_rate,total_under_15_rate,total_over_2_rate,total_under_2_rate,total_over_25_rate,total_under_25_rate,total_over_3_rate,total_under_3_rate,total_over_35_rate,total_under_35_rate,both_team_to_score_yes,both_team_to_score_no,home_scored,away_scored
0,Arsenal,2.044773,4.728864,6.356023,1.280795,2.522159,1.205341,0.883295,10.447727,1.185227,4.966364,1.277500,4.084091,1.627045,2.457386,2.025227,1.961023,2.557727,1.591932,1.723750,2.213636,2.034091,1.034091
1,Aston Villa,3.620244,4.000000,2.884146,1.810244,1.565610,1.290976,1.030488,9.488049,1.240732,4.263902,1.381707,3.315610,1.783171,2.192683,2.336098,1.727805,2.912439,1.465366,1.688780,2.223659,1.536585,1.536585
2,Bournemouth,3.343051,3.779153,2.922203,1.604237,1.508644,1.256610,0.941356,9.224746,1.243390,4.091864,1.386441,3.278644,1.841356,2.101186,2.431525,1.665085,2.992203,1.425932,1.708644,2.208983,1.576271,1.406780
3,Brentford,3.574000,3.594000,2.853000,1.730000,1.503000,1.335000,1.091000,7.315000,1.359000,3.271000,1.603000,2.506000,2.127000,1.802000,3.029000,1.419000,3.752000,1.291000,1.899000,1.931000,1.000000,1.200000
4,Brighton,3.783194,3.741389,2.978194,1.775556,1.536111,1.314444,1.081528,8.144861,1.321250,3.518611,1.551667,2.707500,2.063056,1.883611,2.961528,1.488056,3.581528,1.321250,1.885000,1.960139,1.055556,1.416667


In [47]:
train_data.loc[:,['away_team', 'season', 'away_scored']].groupby(['away_team','season'], as_index=False).sum()

,away_team,season,away_scored
0,Arsenal,2016,13
1,Arsenal,2017,27
2,Arsenal,2018,28
3,Arsenal,2019,18
4,Arsenal,2020,11
...,...,...,...
150,Wolves,2018,11
151,Wolves,2019,19
152,Wolves,2020,14
153,Wolves,2021,16
